# 5 卷积神经网络

- 卷积神经网络是近年来深度学习能在计算机视觉领域取得突破性成果的基石。
- 卷积层、池化层、填充、步幅、输入通道、输出通道。
- AlexNet、VGG、NiN、GoogLeNet、ResNet、NenseNet。
- 批量归一化、残差网络。

## 5.1 二维卷积层

卷积神经网络(convolutional neural network)是含有卷积层(convolutional layer)的神经网络。本章中介绍的卷积神经网络均使用最常见的二维卷积层。它有高和宽两个空间维度，常用来处理图像数据。

### 5.1.1 二维互相关运算

- 虽然卷积层得名于卷积(convolution)运算，但我们通常在卷积层中使用更加直观的互相关(cross-correlation)运算。
- 核数组在卷积计算中又称卷积核或过滤器(filter)。
- 卷积核窗口(又称卷积窗口)的形状取决于卷积核的高和宽。
- 在二维互相关运算中，卷积核窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。

In [2]:
from mxnet import autograd, nd
from mxnet.gluon import nn

def corr2d(X, K): # 本函数已保存在d2lzh包中方便以后使用
    h, w = K.shape
    Y = nd.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [3]:
X = nd.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = nd.array([[0, 1], [2, 3]])
corr2d(X, K)


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>

### 5.1.2 二维卷积层

- 二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏差来得到输出。
- 卷积层的模型参数包括了卷积核和标量偏差。
- 在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差。

实现一个自定义的二维卷积层。

In [4]:
class Conv2D(nn.Block):
    def __init__(self, kernel_size, **kwargs):
        super().__init__(**kwargs)
        self.weight = self.params.get('weight', shape=kernel_size)
        self.bias = self.params.get('bias', shape=(1,))
        
    def forward(self, x):
        return corr2d(x, self.weight.data()) + self.bias.data()

### 5.1.3 图像中物体边缘检测

In [5]:
X = nd.ones((6, 8))
X[:, 2:6] = 0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [6]:
K = nd.array([[1, -1]])

In [7]:
Y = corr2d(X, K)
Y


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

- 卷积层可通过重复使用卷积核有效地表征局部空间。

### 5.1.4 通过数据学习核数组

In [8]:
# 构造一个输出通道数为1，核数组形状为(1, 2)的二维卷积层
conv2d = nn.Conv2D(1, kernel_size=(1, 2))
conv2d.initialize()

# 二维卷积层使用4维输入输出，格式为(样本，通道，高，宽)，这里批量大小和通道数均为1
X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1, 1, 6, 7)

for i in range(10):
    with autograd.record():
        Y_hat = conv2d(X)
        l = (Y_hat - Y) ** 2
    l.backward()
    # 简单起见，这里忽略了偏差
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    if (i + 1) % 2 == 0:
        print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

batch 2, loss 4.949
batch 4, loss 0.831
batch 6, loss 0.140
batch 8, loss 0.024
batch 10, loss 0.004


In [9]:
conv2d.weight.data().reshape((1, 2))


[[ 0.9895    -0.9873705]]
<NDArray 1x2 @cpu(0)>

### 5.1.5 互相关运算和卷积运算

- 实际上，卷积运算与互相关运算类似。
- 为了得到卷积运算的输出，我们只需将核数组左右翻转并上下翻转，再与输入数组做互相关运算。
- 在深度学习中核数组都是学出来的：卷积层无论使用互相关运算或卷积运算都不影响模型预测时的输出。

### 5.1.6 特征图与感受野

- 二维卷积层输出的二维数组可以看做输入在空间维度(高和宽)上某一级的表征，也叫特征图(feature map)。
- 影响元素$x$的前向计算的所有可能输入区域叫做$x$的感受野。
- 我们可以通过更深的卷积神经网络使特征图中单个元素的感受野变得更加广阔，从而捕捉输入上更大尺寸的特征。

### 小结

- 二维卷积层的核心计算是二维互相关运算。再最简单的形式下，它对二维输入数据和卷积核做互相关运算然后加上偏差。
- 我们可以设计卷积核来检测图像中的边缘。
- 我们可以通过数据来学习卷积核。